<!-- -*- mode: markdown; coding: utf-8; fill-column: 60; ispell-dictionary: "english" -*- -->

<meta charset="utf-8"/>
<meta name="viewport" content="width=device-width,initial-scale=1"/>
<link rel="stylesheet" href="style.css">


# EDAF75 - lab 2: Testing the database

As usual we have to tell Jupyter to allow SQL:

In [14]:
%load_ext sql

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


And then we import our movie database

In [11]:
%sql sqlite:///movies.sqlite

We want to make sure that SQLite3 really checks our foreign
key constraints -- to do that, we run:

In [9]:
%%sql
PRAGMA foreign_keys=ON;

 * sqlite:///movies.sqlite
Done.


[]

Now write SQL code for the following tasks:


+ Show the names of all movies.

In [15]:
%%sql
Select *
from ticket

 * sqlite:///movies.sqlite
(sqlite3.OperationalError) no such table: ticket
[SQL: Select *
from ticket]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


+ Show the performance dates for one of the movies.

In [22]:
%%sql
SELECT time_slot.start_time, movies.names
FROM time_slot
JOIN movies ON time_slot.movie_ID = movies.movie_ID
WHERE movies.names = 'Shrek1';


 * sqlite:///movies.sqlite
Done.


start_time,names
February 10 15:00,Shrek1


+ Show all data concerning performances at a given theatere
  on a given date.

In [6]:
%%sql
SELECT *
FROM time_slot
WHERE theater_name = 'Sergelstorg'
AND start_time LIKE 'February 04%';

 * sqlite:///movies.sqlite
(sqlite3.OperationalError) no such table: time_slot
[SQL: SELECT *
FROM time_slot
WHERE theater_name = 'Sergelstorg'
AND start_time LIKE 'February 04%';]
(Background on this error at: https://sqlalche.me/e/20/e3q8)


+ List all customers

In [24]:
%%sql
SELECT * FROM customer;

 * sqlite:///movies.sqlite
Done.


username,fullname,pwd
Lucky on da beat,Luc Sommerland,lucluc12
Joar RR,Joar Rinaldo-Roos,falcon18
Jeppelito,Jesper Lundqvist,123


+ List all tickets

In [26]:
%%sql
SELECT * FROM ticket;

 * sqlite:///movies.sqlite
Done.


screening_ID,ticket_ID,username
1000,9a5f073610e445294da0b20b9f8bebc0,Lucky on da beat
1000,97257a0304232883616b61fadaf81898,Joar RR
3000,519c016d25a12761e9b6519e891ba93a,Jeppelito


+ Create a new ticket to some performance (i.e., insert a
  new row in your table of tickets).

In [27]:
%%sql
INSERT INTO ticket (username, screening_ID)
VALUES ('Jeppelito', 2000);

 * sqlite:///movies.sqlite
1 rows affected.


[]

In newer versions of SQLite (since version 3.35, released
  in March 2021), and in
  [PostgreSQL](https://www.postgresql.org/docs/current/sql-insert.html),
  we can get any value generated during an insert using the
  `INSERT...-RETURNING` statement:

~~~{.sql}
INSERT
INTO       students
VALUES     ('Amy', 3.9, 1200)
RETURNING  s_id
~~~


which would return the generated `s_id` for the new
  student.

  If your SQLite version is older than 3.35, and you can't
  upgrade, you can instead use the following idea: each row
  in a SQLite3 table has a `rowid` attribute, it is a unique
  integer which essentially tells in which order the rows
  were inserted, and it's not displayed in queries unless we
  ask for it. SQLite3 also have a function,
  `last_insert_rowid()`, which returns the `rowid` of the
  last inserted row of a table, so we can see the `s_id` of
  the most recently inserted student with the following
  query:

~~~{.sql}
SELECT s_id
FROM   students
WHERE  rowid = last_insert_rowid();
~~~


Now, check what ticket number we got for the ticket we
  created above (it should be the same as the ticket id,
  which should be a `randomblob`):

In [29]:
%%sql
SELECT ticket_ID, username
FROM ticket
WHERE username = 'Jeppelito' AND screening_ID = 2000;

 * sqlite:///movies.sqlite
Done.


ticket_ID,username
7b483bfecd147c3fdd4888fe5406e3e1,Jeppelito


+ Try to insert two movie theaters with the same name (this
  should fail).

In [30]:
%%sql
INSERT INTO theaters(theater_name, capacity)
VALUES ('DuplicateTheater', 100);

INSERT INTO theaters(theater_name, capacity)
VALUES ('DuplicateTheater', 200);


 * sqlite:///movies.sqlite
1 rows affected.
(sqlite3.IntegrityError) UNIQUE constraint failed: theaters.theater_name
[SQL: INSERT INTO theaters(theater_name, capacity)
VALUES ('DuplicateTheater', 200);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


+ Try to insert a performance where the theater doesnâ€™t
  exist in the database (this should fail).

In [31]:
%%sql
INSERT INTO time_slot (movie_ID, theater_name, start_time, screening_ID)
VALUES ('tt001', 'NonExistentTheater', 'February 12 20:00', 4000);

 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO time_slot (movie_ID, theater_name, start_time, screening_ID)
VALUES ('tt001', 'NonExistentTheater', 'February 12 20:00', 4000);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)


+ Create a ticket where either the user or the performance
  doesnâ€™t exist (this should fail).

In [32]:
%%sql
INSERT INTO ticket (username, screening_ID)
VALUES ('Noa', 1000); 


 * sqlite:///movies.sqlite
(sqlite3.IntegrityError) FOREIGN KEY constraint failed
[SQL: INSERT INTO ticket (username, screening_ID)
VALUES ('Noa', 1000);]
(Background on this error at: https://sqlalche.me/e/20/gkpj)
